![image](https://drive.google.com/u/0/uc?id=15DUc09hFGqR8qcpYiN1OajRNaASmiL6d&export=download)

# **Taller No. 8 - ISIS4825**
## **Proceso de Aprendizaje Automático e Introducción a la Clasificación**
## **Contenido**
1. [**Objetivos**](#id1)
2. [**Problema**](#id2)
3. [**Importando las librerías necesarias para el laboratorio**](#id3)
4. [**Visualización y Análisis Exploratorio**](#id4)
5. [**Preparación de los Datos**](#id5)
6. [**Modelamiento**](#id6)
7. [**Predicción**](#id7)
8. [**Validación**](#id8)
9. [**Trabajo Asíncrono**](#id9)

## **Objetivos**<a name="id1"></a>
- Familiarizarse con las librerías de Scikit-Learn y con el algoritmo de KNN
- Resolver un problema de clasificación multiclase y tomar métricas de desempeño sobre este

## **Problema**<a name="id2"></a>
- En una tienda de ropa buscan crear un algoritmo de clasificación que asigne una etiqueta a 10 tipos de prendas distintas. Desde ropa hasta accesorios.

## **Notebook Configuration**

In [ ]:
!shred -u setup_colab.py
!wget -q "https://github.com/jpcano1/ISIS_4825_Imagenes_Vision/raw/main/Machine%20Learning/setup_colab.py" -O setup_colab.py
import setup_colab as setup
setup.setup_workshop_8()

## **Importando las librerías necesarias para el laboratorio**<a name="id3"></a>

In [ ]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import (train_test_split, ShuffleSplit, 
                                     cross_val_score, GridSearchCV)
from sklearn.metrics import (precision_score, recall_score, confusion_matrix, 
                             accuracy_score, f1_score, roc_curve, 
                             precision_recall_curve)

import utils.general as gen

from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns

## **Visualización y Análisis Exploratorio**<a name="id4"></a>
- Vamos a hacer uso del Dataset `Fashion-MNIST` que consta de 10 clases:
    0. T-Shirt/Top
    1. Trouser
    2. Pullover
    3. Dress
    4. Coat
    5. Sandal
    6. Shirt 
    7. Sneaker
    8. Bag
    9. Ankle Boot
- De igual forma, el dataset tiene 70.000 imágenes en escala de grises con resolución 28x28. Sin embargo, las imágenes ya se encuentran aplanadas con tamaño de vector 784 componentes.

In [ ]:
fashion_mnist = datasets.fetch_openml("Fashion-MNIST")

In [ ]:
data, target = fashion_mnist.data, fashion_mnist.target

In [ ]:
data.shape, target.shape

In [ ]:
random_sample = np.random.choice(np.arange(len(data)), 9)
gen.visualize_subplot(
    data[random_sample].reshape(-1, 28, 28),
    target[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(data)), 9)
gen.visualize_subplot(
    data[random_sample].reshape(-1, 28, 28),
    target[random_sample],  (3, 3), (6, 6)
)

In [ ]:
target_classes = ["T-Shirt/Top", "Trouser", "Pullover", "Dress", 
                  "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

In [ ]:
target_distribution = pd.Series(target).value_counts().sort_index()
target_distribution.index = target_classes

In [ ]:
target_distribution

## **Preparación de los Datos**<a name="id5"></a>
- Dado que es un algoritmo que requiere vectores, vamos a necesitar que todas nuestras imágenes sean aplanadas, si es que aún no lo están.

### **Tratamiento de Imágenes**
- En este caso, nuestras imágenes ya son vectores, sin embargo vamos a ver cómo hacer su transformación vector-imagen e imagen-vector.

In [ ]:
sample_img = data[0].reshape(28, 28)
sample_target = target[0]

In [ ]:
gen.imshow(sample_img, color=False)

In [ ]:
sample_img.shape

In [ ]:
sample_img = sample_img.flatten()

In [ ]:
sample_img.shape

In [ ]:
sample_img.min()

In [ ]:
sample_img.max()

### **Train Set, Validation Set, Test Set**
- Generalmente, en el mundo del computer vision, se hace la siguiente partición de datasets:
    - Train Data:
        - Train Set
        - Validation Set
    - Test Data:
        - Test Set
- La partición de los datasets la podemos hacer de varias formas, pero en esta ocasión veremos la partición por índices y por contenido.

#### **Partición por Índice**
- Buscamos dividir nuestro dataset a partir de sus índices.

In [ ]:
rnd_data = np.random.choice(np.arange(len(data)), 10000)
full_data = data.copy()
full_target = target.copy()
data = data[rnd_data]
target = target[rnd_data]

- Aquí usamos el `random_state` para definir una semilla de aleatoriedad para que los grupos generados se mantengan siempre.

In [ ]:
ss_full_train_test = ShuffleSplit(n_splits=10, test_size=0.2, random_state=1234)

In [ ]:
for full_train_index, test_index in ss_full_train_test.split(data):
    pass

In [ ]:
full_train_index

In [ ]:
test_index

In [ ]:
full_train_set, test_set = ((data[full_train_index], target[full_train_index]), 
                            (data[test_index], target[test_index]))

In [ ]:
ss_train_val = ShuffleSplit(n_splits=10, test_size=0.2, random_state=5678)

In [ ]:
for train_index, val_index in ss_train_val.split(full_train_set[0]):
    pass

In [ ]:
train_set, val_set = ((full_train_set[0][train_index], full_train_set[1][train_index]), 
                      (full_train_set[0][val_index], full_train_set[1][val_index]))

In [ ]:
X_train, y_train = train_set[0], train_set[1]
X_val, y_val = val_set[0], val_set[1]
X_test, y_test = test_set[0], test_set[1]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_val.shape, y_val.shape

In [ ]:
X_test.shape, y_test.shape

- Generemos una muestra de imágenes por cada set generado.

In [ ]:
random_sample = np.random.choice(np.arange(len(X_train)), 9)
gen.visualize_subplot(
    X_train[random_sample].reshape(-1, 28, 28),
    y_train[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_val)), 9)
gen.visualize_subplot(
    X_val[random_sample].reshape(-1, 28, 28),
    y_val[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_test)), 9)
gen.visualize_subplot(
    X_test[random_sample].reshape(-1, 28, 28),
    y_test[random_sample],  (3, 3), (6, 6)
)

#### **Partición por Contenido**
- Aquí no buscamos partir nuestro dataset a partir de los índices que contiene, sino por el cuerpo de la data.

In [ ]:
full_X_train, X_test, full_y_train, y_test = train_test_split(data, target, 
                                                              test_size=0.2, 
                                                              random_state=1234)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(full_X_train, full_y_train, 
                                                  test_size=0.2, 
                                                  random_state=1234)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_train)), 9)
gen.visualize_subplot(
    X_train[random_sample].reshape(-1, 28, 28),
    y_train[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_val)), 9)
gen.visualize_subplot(
    X_val[random_sample].reshape(-1, 28, 28),
    y_val[random_sample],  (3, 3), (6, 6)
)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_test)), 9)
gen.visualize_subplot(
    X_test[random_sample].reshape(-1, 28, 28),
    y_test[random_sample],  (3, 3), (6, 6)
)

## **Modelamiento**<a name="id6"></a>
- A la hora de modelar los datos, buscamos un algoritmo que generalice la forma como los datos se comportan y con base en ello, pueda generar predicciones.

### **K-Nearest-Neighbors**
- En este caso, vamos a utilizar un algoritmo de modelado no lineal basado en vecindades o *neighborhoods*. Se trata de *K-Nearest Neighbors*.

![image](https://miro.medium.com/max/3544/1*4F-q86XFr2-EsaAcz0Zu5A.png)

> Tomado de [Towards Data Science](https://towardsdatascience.com/k-nearest-neighbor-python-2fccc47d2a55)

- Este espacio lo tomamos, generalmente, para buscar algoritmos que puedan ser usados para modelar, y una vez encontrados, exploramos los hiperparámetros que podamos usar para mejorar los resultados de nuestras predicciones.

In [ ]:
KNeighborsClassifier?

- En este caso, se utiliza el valor de vecinos por defecto, que es el `k=5`

In [ ]:
knn_clf = KNeighborsClassifier(n_jobs=-1)

In [ ]:
knn_clf.fit(X_train, y_train)

## **Predicción**<a name="id7"></a>
- En esta etapa nos concentramos en hacer nuestras predicciones y validarlas con el ojo.

In [ ]:
random_sample = np.random.choice(np.arange(len(X_test)), 9)
y_pred = knn_clf.predict(X_test[random_sample])

In [ ]:
gen.visualize_subplot(
    X_test[random_sample].reshape(-1, 28, 28),
    y_pred, (3, 3), (6, 6)
)

## **Validación**<a name="id8"></a>
- En esta etapa de evaluación realizamos el proceso de toma de métricas. Por lo tanto, dado que estamos resolviendo un problema de clasificación, vamos a usar la matriz de confusión y las siguientes métricas:
    - Accuracy score: $\frac{TP + TN}{TP + TN + FP + FN}$
    - Precision: $\frac{TP}{TP + FP}$
    - Cobertura: $\frac{TP}{TP + FN}$ (Recall, Sensitivity)
    - F1 score: $\frac{TP}{TP + \frac{FN + FP}{2}}$ (Harmonic Mean)

In [ ]:
y_pred = knn_clf.predict(X_test)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
pd.DataFrame(conf_matrix)

- En esta matriz de confusión podemos ver claramente que las diagonales sobresalen, es decir, hubo un gran número de predicciones correctas.

In [ ]:
plt.matshow(conf_matrix, cmap="gray")
plt.grid(0)
plt.show()

In [ ]:
np.trace(conf_matrix)

In [ ]:
norm_conf_mat = conf_matrix / conf_matrix.sum(axis=1, keepdims=True)
np.fill_diagonal(norm_conf_mat, 0)

- Aquí vemos que, aunque nuestro algoritmo clasificó correctamente alrededor del 80% de nuestro dataset, siempre las clasificaciones erróneas fueron bastantes.

In [ ]:
plt.matshow(norm_conf_mat, cmap="gray")
plt.grid(0)
plt.show()

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred, average="weighted")

In [ ]:
recall_score(y_test, y_pred, average="weighted")

In [ ]:
f1_score(y_test, y_pred, average="weighted")

In [ ]:
cross_val_score(knn_clf, full_X_train, full_y_train, cv=4, scoring="accuracy")

- En esta etapa de nuestro workflow, vamos a probar valores de *k* que mejoren nuestros resultados.

In [ ]:
rec_scores = {}
prec_scores = {}
for k in tqdm(range(5, 16, 5)):
    knn_clf = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    knn_clf.fit(X_train, y_train)
    y_pred = knn_clf.predict(X_val)
    rec_scores[k] = recall_score(y_val, y_pred, average="weighted")
    prec_scores[k] = precision_score(y_val, y_pred, average="weighted")
    print(f"------Number of Neighbors: {k}--------")
    print(f"Recall Score: {rec_scores[k]}")
    print(f"Precision Score: {prec_scores[k]}")

In [ ]:
total_data = {
    "neighbors": list(range(5, 16, 5)),
    "rec_scores": list(rec_scores.values()),
    "prec_scores": list(prec_scores.values())
}

total_df = pd.DataFrame(total_data)

In [ ]:
plt.plot(total_df["neighbors"], total_df["rec_scores"])
plt.xlabel("K Neighbors")
plt.ylabel("Recall")
plt.show()

In [ ]:
plt.plot(total_df["neighbors"], total_df["prec_scores"])
plt.xlabel("K Neighbors")
plt.ylabel("Precision")
plt.show()

In [ ]:
knn_best = KNeighborsClassifier(n_jobs=-1)
knn_best.fit(X_train, y_train)

In [ ]:
random_sample = np.random.choice(np.arange(len(X_test)), 9)
y_pred = knn_best.predict(X_test[random_sample])
gen.visualize_subplot(
    X_test[random_sample].reshape(-1, 28, 28),
    y_pred, (3, 3), (6, 6)
)

## **Trabajo Asíncrono**<a name="id9"></a>

- Con el mismo conjunto de imágenes, o con el de mnist de dígitos, hacer una partición del dataset que garatice que se mantienen las proporciones de cada clase por cada partición. Para esto, usarán [`StratifiedShufflesplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html). Para acceder al conjunto de dígitos de mnist, utilizar la siguiente línea de código: `mnist = datasets.fetch_openml("mnist_784", version=1)`
- Luego, construir una gráfica que muestre cómo varía el rendimiento sobre entrenamiento y validación a medida que aumenta el valor de k. A partir de esta gráfica, mostrar el rendimiento sobre el test set con el valor de k seleccionado.
- A continuación, utilizar [`GridSearch`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para determinar los mejores valores de los hiperparámetros. Para eso, averiguar sobre los siguientes hiperparámetros:
    - `n_neighbors`
    - `weights`
    - `algorithm`
- Por último, mostrar algunas imágenes del conjunto test con la clase estimada por el mejor clasificador obtenido en el punto anterior y mostrarlos en una cuadrícula.